In [ ]:

%pip install sagemaker


In [ ]:
import boto3
import numpy as np
import os
import pandas as pd
import re
import sagemaker
from sagemaker.tensorflow import TensorFlowModel
from sagemaker.utils import S3DataConfig
from sagemaker.tensorflow import TensorFlow
import shutil
import tarfile
import tensorflow as tf
from tensorflow.python.keras.utils.np_utils import to_categorical
from keras.layers import Input, Dense

import zipfile


role = "arn:aws:iam::454075184092:role/image_classification_role"
sm_session = sagemaker.Session()


In [ ]:
import sagemaker.s3 as s3

# Specify the S3 bucket and key (path to the file in the bucket)
bucket_name = 'kagglebird'
file_key = 'archive.zip'

# Define the local file path where you want to download the file
local_file_path = 'root'

# Use the S3Downloader to download the file from S3 to the local path
s3.S3Downloader.download(s3_uri=f's3://{bucket_name}/{file_key}', local_path=local_file_path)



In [ ]:

# Specify the path to the zip file
zip_file_path = 'root/archive.zip'

# Specify the path to the new data directory
data_dir = 'data'

# Create the new data directory if it doesn't exist
os.makedirs(data_dir, exist_ok=True)

# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all contents to the new data directory
    zip_ref.extractall(data_dir)

print(f"Unzipped contents to {data_dir}")

In [6]:
s3_data = sm_session.upload_data(path='data', bucket="kagglebird", key_prefix='newdata')
# s3_data = sm_session.upload_data(path='data/train', bucket="kagglebird", key_prefix='train')
# s3_data = sm_session.upload_data(path='data/valid', bucket="kagglebird", key_prefix='valid')
# s3_data = sm_session.upload_data(path='data/train', bucket="kagglebird", key_prefix='test')

In [ ]:
# Specify the script, role, instance type, and hyperparameters
script_path = 'train.py'
role = role
instance_type = 'ml.m4.xlarge'
hyperparameters = {'epochs': 40, 'learning-rate': 0.001, 'batch-size': 32, "training": "./data/test", "validation": "./data/valid"}

# Create TensorFlow estimator
estimator = TensorFlow(
    entry_point=script_path,
    role=role,
    instance_count=1,
    instance_type=instance_type,
    framework_version='2.13',  
    py_version='py310',
    hyperparameters=hyperparameters
)

# Train the model
estimator.fit()


In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")